In [5]:
## Stacking classifier 
## using sklearn and mlxtend
## fitted models are stored for predicting unseen test data

## Author: Yichao Li
## Last modified: 9-3-2017

## Usage:
## from stacking_clf import mixer_clf
## model = mixer_clf(X,y)
## feature selection method is not included
## parameter tuning is done using random search

## Note: class_weight={0:.3, 1:.7} is pre-assigned

import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, auc, f1_score, matthews_corrcoef
from sklearn.metrics import precision_recall_curve, roc_auc_score
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression,RidgeClassifierCV,SGDClassifier,SGDRegressor
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor,RadiusNeighborsRegressor
from sklearn.neural_network import MLPClassifier,MLPRegressor
from sklearn.naive_bayes import GaussianNB 
from sklearn.gaussian_process import GaussianProcessClassifier,GaussianProcessRegressor
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from mlxtend.classifier import StackingCVClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
import numpy as np
import xgboost as xgb
import pickle

# from sklearn.isotonic import IsotonicRegression
from sklearn.svm import SVR
from sklearn.linear_model import Lasso,ElasticNet
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor,ExtraTreesRegressor
from sklearn.linear_model import Ridge,LinearRegression,HuberRegressor,PassiveAggressiveRegressor,BayesianRidge,ARDRegression,Lars,LassoLars,LassoLarsIC,MultiTaskLasso,OrthogonalMatchingPursuit,RANSACRegressor,TheilSenRegressor
from mlxtend.regressor import StackingCVRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error

def mixer_reg(X,y,RANDOM_SEED = 42,n_iter=3):
	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=RANDOM_SEED)
	params={}
	ridge = Ridge()
	params['ridge__alpha'] = [0.5,1,5,10]
	lasso = Lasso(selection ="random",alpha=1.0)
	params['lasso__alpha'] = [0.5,1,5,10]
	rf = RandomForestRegressor(n_estimators=50,criterion="mae",max_features='auto')
	# ENet = ElasticNet(alpha=1,l1_ratio=0.3)
	# params['elasticnet__alpha'] = [0.5,1,5,10]
	# params['elasticnet__l1_ratio'] = [0.3,0.5,0.8]
	# GBR = GradientBoostingRegressor(loss="ls",max_depth=3)
	# params['gradientboostingregressor__loss'] = ["ls","lad","huber","quantile"]
	# params['gradientboostingregressor__max_depth'] = [3,8,20]
	# ETR = ExtraTreesRegressor(n_estimators=50,criterion="mae")
	# KNNR = KNeighborsRegressor(n_neighbors=20,p=2)
	# params['kneighborsregressor__n_neighbors'] = [5,20]
	# SVRR = SVR(kernel = "linear",C=1)
	# LR = LinearRegression()
	# HR = HuberRegressor()
	# params['huberregressor__epsilon'] = [1.1,1.35,3]
	# PAR1 = PassiveAggressiveRegressor(loss="epsilon_insensitive")
	# PAR2 = PassiveAggressiveRegressor(loss="squared_epsilon_insensitive")
	# BayesR = BayesianRidge()
	# ARDR = ARDRegression()
	# LARSR = Lars()
	# LassoLarsR = LassoLars()
	# LassoLarsICR = LassoLarsIC()
	# OrthogonalMatchingPursuitR = OrthogonalMatchingPursuit()
	# RANSACRegressorR = RANSACRegressor()
	# SGDR = SGDRegressor()
	# TheilSenRegressorR = TheilSenRegressor()
	# GaussianProcessRegressorR = GaussianProcessRegressor()
	# ada_ridge = AdaBoostRegressor(base_estimator=Ridge())
	# ada_KNN = AdaBoostRegressor(base_estimator=KNeighborsRegressor(n_neighbors=5))
	# ada_lasso = AdaBoostRegressor(base_estimator=Lasso())
	# ada_DT = AdaBoostRegressor()
	# ada_HR = AdaBoostRegressor(base_estimator=HuberRegressor(epsilon = 1.35,alpha =0.01))
	# ada_EN = AdaBoostRegressor(base_estimator=ElasticNet())
	# ada_LR = AdaBoostRegressor(base_estimator=LinearRegression())
	xgbR = xgb.sklearn.XGBRegressor(n_estimators=500,nthread=1)
# 	my_reg_list = [ridge,lasso,rf,ENet,GBR,ETR,KNNR,SVRR,LR,HR,PAR1,PAR2,BayesR,ARDR,LARSR,LassoLarsR,LassoLarsICR,OrthogonalMatchingPursuitR,RANSACRegressorR,SGDR,TheilSenRegressorR,GaussianProcessRegressorR,ada_ridge,ada_KNN,ada_lasso,ada_DT,ada_HR,ada_EN,ada_LR,xgbR]
	my_reg_list = [ridge,lasso,rf,xgbR]
	mix_CV_regressor = StackingCVRegressor(regressors=my_reg_list, meta_regressor=Lasso(),use_features_in_secondary = False,cv = 2)	
	params['meta-lasso__alpha'] = [0.01, 0.1 ,0.5, 1.0 ,5, 10.0]
	# params['meta-xgbregressor__max_depth'] = [1,3,5,7,10,15,50]
	# params['meta-xgbregressor__learning_rate'] = [0.1,0.01,1,10]
	# params['meta-xgbregressor__gamma'] = [0.1,0.01,0.5,0.8]
	# params['meta-xgbregressor__min_child_weight'] = [1,5,10,20]
	# params['meta-xgbregressor__max_delta_step'] = [1,5,10,20]
	# params['meta-xgbregressor__reg_alpha'] = [0.1,0.01,1,10]
	# params['meta-xgbregressor__reg_lambda'] = [0.1,0.01,1,10]
	
	grid = RandomizedSearchCV(estimator=mix_CV_regressor, param_distributions=params, cv=2,n_iter=n_iter,refit=True,n_jobs=-1,scoring="neg_mean_absolute_error",verbose=10)						
	grid.fit(X_train, y_train)
	print('Best parameters: %s' % grid.best_params_)
	print('CV best MAE: %.7f' % grid.best_score_)
	best_clf = grid.best_estimator_
	predict_value = best_clf.predict(X_test)
	MAE = mean_absolute_error(y_test,predict_value)
	MSE = mean_squared_error(y_test,predict_value)
	print('testing MAE: %.7f' % MAE)
	print('testing MSE: %.7f' % MSE)
	return best_clf
	

def mixer_reg_bk(X,y,RANDOM_SEED = 42,n_iter=3):
	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=RANDOM_SEED)
	params={}
	ridge = Ridge()
	params['ridge__alpha'] = [0.5,1,5,10]
	# ridge2 = Ridge(alpha=0.1)
	# ridge3 = Ridge(alpha=10)
	lasso = Lasso(selection ="random",alpha=1.0)
	params['lasso__alpha'] = [0.5,1,5,10]
	# lasso2 = Lasso(selection ="random",alpha=0.1)
	# lasso3 = Lasso(selection ="random",alpha=10)
	# lasso4 = Lasso(selection ="cyclic",alpha=1.0)
	# lasso5 = Lasso(selection ="cyclic",alpha=0.1)
	# lasso6 = Lasso(selection ="cyclic",alpha=10)
	# rf1 = RandomForestRegressor(n_estimators=50,criterion="mse",max_features=500)
	# rf2 = RandomForestRegressor(n_estimators=50,criterion="mse",max_features=300)
	# rf3 = RandomForestRegressor(n_estimators=50,criterion="mse",max_features=100)
	# rf4 = RandomForestRegressor(n_estimators=50,criterion="mse",max_features='auto')
	# rf5 = RandomForestRegressor(n_estimators=50,criterion="mae",max_features=500)
	# rf6 = RandomForestRegressor(n_estimators=50,criterion="mae",max_features=300)
	# rf7 = RandomForestRegressor(n_estimators=50,criterion="mae",max_features=100)
	rf = RandomForestRegressor(n_estimators=50,criterion="mae",max_features='auto')
	# ENet1 = ElasticNet(alpha=1,l1_ratio=0.3)
	ENet = ElasticNet(alpha=1,l1_ratio=0.3)
	params['elasticnet__alpha'] = [0.5,1,5,10]
	params['elasticnet__l1_ratio'] = [0.3,0.5,0.8]
	# ENet2 = ElasticNet(alpha=1,l1_ratio=0.5)
	# ENet3 = ElasticNet(alpha=1,l1_ratio=0.8)
	# ENet4 = ElasticNet(alpha=10,l1_ratio=0.3)
	# ENet5 = ElasticNet(alpha=10,l1_ratio=0.5)
	# ENet6 = ElasticNet(alpha=10,l1_ratio=0.8)
	# ENet7 = ElasticNet(alpha=0.1,l1_ratio=0.3)
	# ENet8 = ElasticNet(alpha=0.1,l1_ratio=0.5)
	# ENet9 = ElasticNet(alpha=0.1,l1_ratio=0.8)
	GBR = GradientBoostingRegressor(loss="ls",max_depth=3)
	params['gradientboostingregressor__loss'] = ["ls","lad","huber","quantile"]
	params['gradientboostingregressor__max_depth'] = [3,8,20]
	# GBR1 = GradientBoostingRegressor(loss="ls",max_depth=3)
	# GBR2 = GradientBoostingRegressor(loss="lad",max_depth=3)
	# GBR3 = GradientBoostingRegressor(loss="huber",max_depth=3)
	# GBR4 = GradientBoostingRegressor(loss="quantile",max_depth=3)
	# GBR5 = GradientBoostingRegressor(loss="ls",max_depth=6)
	# GBR6 = GradientBoostingRegressor(loss="lad",max_depth=6)
	# GBR7 = GradientBoostingRegressor(loss="huber",max_depth=6)
	# GBR8 = GradientBoostingRegressor(loss="quantile",max_depth=6)
	# GBR9 = GradientBoostingRegressor(loss="ls",max_depth=20)
	# GBR10 = GradientBoostingRegressor(loss="lad",max_depth=20)
	# GBR11 = GradientBoostingRegressor(loss="huber",max_depth=20)
	# GBR12 = GradientBoostingRegressor(loss="quantile",max_depth=20)
	# ETR1 = ExtraTreesRegressor(n_estimators=50)
	ETR = ExtraTreesRegressor(n_estimators=50,criterion="mae")
	# KNNR1 = KNeighborsRegressor(n_neighbors=5,p=1)
	# KNNR2 = KNeighborsRegressor(n_neighbors=20,p=1)
	# KNNR3 = KNeighborsRegressor(n_neighbors=5,p=2)
	# KNNR4 = KNeighborsRegressor(n_neighbors=20,p=2)
	KNNR = KNeighborsRegressor(n_neighbors=20,p=2)
	params['kneighborsregressor__n_neighbors'] = [5,20]
	# MLPR = MLPRegressor()
	# SVR1 = SVR(kernel = "linear",C=100)
	SVRR = SVR(kernel = "linear",C=1)
	# SVR3 = SVR(kernel = "linear",C=0.01)
	# SVR4 = SVR()
	LR = LinearRegression()
	# HR1 = HuberRegressor(epsilon = 1.35,alpha =0.01)
	HR = HuberRegressor()
	params['huberregressor__epsilon'] = [1.1,1.35,3]
	# HR2 = HuberRegressor(epsilon = 2,alpha =0.01)
	# HR3 = HuberRegressor(epsilon = 3,alpha =0.01)
	# HR4 = HuberRegressor(epsilon = 1.1,alpha =0.01)
	# HR5 = HuberRegressor(epsilon = 1.35,alpha =0.0001)
	# HR6 = HuberRegressor(epsilon = 2,alpha =0.0001)
	# HR7 = HuberRegressor(epsilon = 3,alpha =0.0001)
	# HR8 = HuberRegressor(epsilon = 1.1,alpha =0.0001)
	PAR1 = PassiveAggressiveRegressor(loss="epsilon_insensitive")
	PAR2 = PassiveAggressiveRegressor(loss="squared_epsilon_insensitive")
	BayesR = BayesianRidge()
	ARDR = ARDRegression()
	LARSR = Lars()
	LassoLarsR = LassoLars()
	LassoLarsICR = LassoLarsIC()
	# MultiTaskLassoR = MultiTaskLasso()
	OrthogonalMatchingPursuitR = OrthogonalMatchingPursuit()
	RANSACRegressorR = RANSACRegressor()
	SGDR = SGDRegressor()
	TheilSenRegressorR = TheilSenRegressor()
	GaussianProcessRegressorR = GaussianProcessRegressor()
	# IsoR = IsotonicRegression()
	# RNR1 = RadiusNeighborsRegressor(radius=1.0)
	# RNR2 = RadiusNeighborsRegressor(radius=5)
	# RNR3 = RadiusNeighborsRegressor(radius=10)
	ada_ridge = AdaBoostRegressor(base_estimator=Ridge())
	# ada_SVR = AdaBoostRegressor(base_estimator=SVR(kernel = "linear",C=1))
	ada_KNN = AdaBoostRegressor(base_estimator=KNeighborsRegressor(n_neighbors=5))
	ada_lasso = AdaBoostRegressor(base_estimator=Lasso())
	ada_DT = AdaBoostRegressor()
	ada_HR = AdaBoostRegressor(base_estimator=HuberRegressor(epsilon = 1.35,alpha =0.01))
	ada_EN = AdaBoostRegressor(base_estimator=ElasticNet())
	ada_LR = AdaBoostRegressor(base_estimator=LinearRegression())
	xgbR = xgb.sklearn.XGBRegressor(n_estimators=1000)
	# my_reg_list = [ridge1,ridge2,ridge3,lasso1,lasso2,lasso3,lasso4,lasso5,lasso6,rf1,rf2,rf3,rf4,rf5,rf6,rf7,rf8,ENet1,ENet2,ENet3,ENet4,ENet5,ENet6,ENet7,ENet8,ENet9,GBR1,GBR2,GBR3,GBR4,GBR5,GBR6,GBR7,GBR8,GBR9,GBR10,GBR11,GBR12,ETR1,ETR2,KNNR1,KNNR2,KNNR3,KNNR4,MLPR,SVR1,SVR2,SVR3,SVR4,LR,HR1,HR2,HR3,HR4,HR5,HR6,HR7,HR8,PAR1,PAR2,BayesR,ARDR,LARSR,LassoLarsR,LassoLarsICR,MultiTaskLassoR,OrthogonalMatchingPursuitR,RANSACRegressorR,SGDR,TheilSenRegressorR,GaussianProcessRegressorR,IsoR,RNR1,RNR2,RNR3,ada_ridge,ada_SVR,ada_KNN,ada_lasso,ada_DT,ada_HR,ada_EN,ada_LR]
	# my_reg_list = [ridge1,ridge2,ridge3,lasso1,lasso2,lasso3,lasso4,lasso5,lasso6,rf4,rf8,ENet1,ENet2,ENet3,ENet4,ENet5,ENet6,ENet7,ENet8,ENet9,GBR1,GBR2,GBR3,GBR4,GBR5,GBR6,GBR7,GBR8,GBR9,GBR10,GBR11,GBR12,ETR1,ETR2,KNNR1,KNNR2,KNNR3,KNNR4,MLPR,SVR2,SVR3,SVR4,LR,HR1,HR2,HR3,HR4,HR5,HR6,HR7,HR8,PAR1,PAR2,BayesR,ARDR,LARSR,LassoLarsR,LassoLarsICR,OrthogonalMatchingPursuitR,RANSACRegressorR,SGDR,TheilSenRegressorR,GaussianProcessRegressorR,RNR1,RNR2,RNR3,ada_ridge,ada_KNN,ada_lasso,ada_DT,ada_HR,ada_EN,ada_LR]
	my_reg_list = [ridge,lasso,rf,ENet,GBR,ETR,KNNR,SVRR,LR,HR,PAR1,PAR2,BayesR,ARDR,LARSR,LassoLarsR,LassoLarsICR,OrthogonalMatchingPursuitR,RANSACRegressorR,SGDR,TheilSenRegressorR,GaussianProcessRegressorR,ada_ridge,ada_KNN,ada_lasso,ada_DT,ada_HR,ada_EN,ada_LR,xgbR]
	# my_reg_list = [ridge1,xgbR,ridge3,lasso1]
	mix_CV_regressor = StackingCVRegressor(regressors=my_reg_list, meta_regressor=Lasso(),use_features_in_secondary = False,cv = 2)	
	
	params['meta-lasso__alpha'] = [0.01, 0.1 ,0.5, 1.0 ,5, 10.0]
	# params['xgbregressor__max_depth'] = [1,3,5,7,10,15,50]
	# params['xgbregressor__learning_rate'] = [0.1,0.01,1,10]
	# params['xgbregressor__gamma'] = [0.1,0.01,0.5,0.8]
	# params['xgbregressor__min_child_weight'] = [1,5,10,20]
	# params['xgbregressor__max_delta_step'] = [1,5,10,20]
	# params['xgbregressor__reg_alpha'] = [0.1,0.01,1,10]
	# params['xgbregressor__reg_lambda'] = [0.1,0.01,1,10]
	# params['meta-xgbregressor__max_depth'] = [1,3,5,7,10,15,50]
	# params['meta-xgbregressor__learning_rate'] = [0.1,0.01,1,10]
	# params['meta-xgbregressor__gamma'] = [0.1,0.01,0.5,0.8]
	# params['meta-xgbregressor__min_child_weight'] = [1,5,10,20]
	# params['meta-xgbregressor__max_delta_step'] = [1,5,10,20]
	# params['meta-xgbregressor__reg_alpha'] = [0.1,0.01,1,10]
	# params['meta-xgbregressor__reg_lambda'] = [0.1,0.01,1,10]
	grid = RandomizedSearchCV(estimator=mix_CV_regressor, param_distributions=params, cv=2,n_iter=n_iter,refit=True,n_jobs=-1,scoring="neg_mean_absolute_error",verbose=10)						
	grid.fit(X_train, y_train)
	print('Best parameters: %s' % grid.best_params_)
	print('CV best MAE: %.7f' % grid.best_score_)
	best_clf = grid.best_estimator_
	predict_value = best_clf.predict(X_test)
	MAE = mean_absolute_error(y_test,predict_value)
	MSE = mean_squared_error(y_test,predict_value)
	print('testing MAE: %.7f' % MAE)
	print('testing MSE: %.7f' % MSE)
	return best_clf
	

def scikitlearn_calc_auPRC(y_true, y_score):  
	precision, recall, _ = precision_recall_curve(y_true, y_score)
	return auc(recall, precision)

def Balanced_ACC(y_true,y_pred,wrt=1):
	tp = 0.0
	tn = 0.0
	correct = 0.0
	y_true = list(y_true)
	if len(y_pred) != len(y_true):
		print "len(y_pred) != len(y_true)!"
		exit()
	for i in range(len(y_pred)):
		if y_pred[i] == y_true[i]:
			if y_pred[i] == wrt:
				tp += 1
			else:
				tn += 1
			correct += 1
	total = len(y_true)
	T = y_true.count(wrt)
	N = total - T
	return (tp/T+tn/N)/2

def mixer_clf(X,y,RANDOM_SEED = 42,n_iter=50):
	## n_iter is the number of parameter settings for RandomizedSearchCV

	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)
	# print y_test
	# print Balanced_ACC(y_test, y_test)
	# exit()
	# Initializing models
	clf1 = KNeighborsClassifier(n_neighbors=5)
	clf1_2 = KNeighborsClassifier(n_neighbors=5,weights="distance")
	clf2 = RandomForestClassifier(random_state=RANDOM_SEED,n_estimators=50,criterion='gini',class_weight={0:.3, 1:.7})
	clf2_2 = RandomForestClassifier(random_state=RANDOM_SEED,n_estimators=50,criterion='entropy',class_weight={0:.3, 1:.7})
	clf12 = ExtraTreesClassifier(random_state=RANDOM_SEED,n_estimators=50,criterion='gini',class_weight={0:.3, 1:.7})
	clf12_2 = ExtraTreesClassifier(random_state=RANDOM_SEED,n_estimators=50,criterion='entropy',class_weight={0:.3, 1:.7})
	clf10 = GradientBoostingClassifier(learning_rate=0.05, subsample=0.8, max_depth=6, n_estimators=50)
	clf3 = GaussianNB()
	clf4 = SVC(probability=True,kernel="linear",class_weight={0:.3, 1:.7})
	clf6 = GaussianProcessClassifier()
	clf7 = MLPClassifier()
	clf8 = AdaBoostClassifier(base_estimator=SVC(probability=True, kernel='linear',class_weight={0:.3, 1:.7}))
	# clf13 = AdaBoostClassifier(base_estimator=RidgeClassifierCV(),algorithm = 'SAMME')
	clf9 = QuadraticDiscriminantAnalysis()
	# clf13 = RidgeClassifierCV()
	clf14 = SGDClassifier(loss="log")
	lr = LogisticRegression(class_weight={0:.3, 1:.7})
	xgboost_clf = xgb.sklearn.XGBClassifier(n_estimators=1000,subsample=0.8,objective= 'binary:logistic')

	np.random.seed(RANDOM_SEED)
	sclf = StackingCVClassifier(classifiers=[clf1, clf1_2,clf2,clf2_2,clf12,clf12_2,clf10,clf3,clf9,lr,clf4,xgboost_clf,clf7,clf8,clf14], 
								meta_classifier=xgboost_clf,use_probas = True,use_features_in_secondary = True)
	params = {'logisticregression__C': [0.01,0.1,0.5,1,5,10],'svc__C':[0.01,0.5,10,100]}
	params['xgbclassifier__learning_rate'] = [0.1,0.5,1]
	params['xgbclassifier__max_depth'] = [5,10,20]
	params['xgbclassifier__min_child_weight'] = [1,5,10]
	params['xgbclassifier__gamma'] = [0,0.1,0.3,0.5,0.8]
	params['xgbclassifier__colsample_bytree'] = [0.1,0.5,0.8]
	params['xgbclassifier__max_delta_step'] = [0,0.1,0.4,0.8]
	# params['meta-svc__C'] = [0.01,0.1,0.5,1,5,10]
	# params['meta-logisticregression__C'] = [0.01,0.1,0.5,1,5,10]
	## tried many meta-classifiers, meta-logisticregression__C is the best
	params['meta-xgbclassifier__learning_rate'] = [0.1,0.5,1]
	params['meta-xgbclassifier__max_depth'] = [5,10,20]
	params['meta-xgbclassifier__min_child_weight'] = [1,5,10]
	params['meta-xgbclassifier__gamma'] = [0,0.1,0.3,0.5,0.8]
	params['meta-xgbclassifier__colsample_bytree'] = [0.1,0.5,0.8]
	# grid = GridSearchCV(estimator=sclf, 
						# param_grid=params, 
						# cv=3,
						# refit=True,n_jobs=-1,scoring="average_precision",verbose=10)
	grid = RandomizedSearchCV(estimator=sclf, 
						param_distributions=params, 
						cv=2,
						n_iter=n_iter,
						refit=True,n_jobs=-1,scoring="average_precision",verbose=10)					

	grid.fit(X_train, y_train)

	# cv_keys = ('mean_test_score', 'std_test_score', 'params')


	print('Best parameters: %s' % grid.best_params_)
	print('CV best auPRC: %.2f' % grid.best_score_)

	best_clf = grid.best_estimator_

	predict_prob = best_clf.predict_proba(X_test)
	predict_prob = np.array(map(lambda x:x[1],predict_prob)).ravel()
	auPRC = scikitlearn_calc_auPRC(y_test, predict_prob)
	auROC = roc_auc_score(y_test, predict_prob)

	Y_pred = best_clf.predict(X_test)
	# print Y_pred
	MCC = matthews_corrcoef(y_test,Y_pred)
	f1 = f1_score(y_test,Y_pred)
	BACC = Balanced_ACC(y_test,Y_pred)


	print('testing auROC: %.2f' % auROC)
	print('testing auPRC: %.2f' % auPRC)
	print('testing MCC: %.2f' % MCC)
	print('testing f1: %.2f' % f1)
	print('testing BACC: %.2f' % BACC)





	return best_clf


def mixer_clf_train_test(X_train,X_test, y_train, y_test,RANDOM_SEED = 42,n_iter=50):
	## n_iter is the number of parameter settings for RandomizedSearchCV

	# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)
	# print y_test
	# print Balanced_ACC(y_test, y_test)
	# exit()
	# Initializing models
	clf1 = KNeighborsClassifier(n_neighbors=5)
	clf1_2 = KNeighborsClassifier(n_neighbors=5,weights="distance")
	clf2 = RandomForestClassifier(random_state=RANDOM_SEED,n_estimators=50,criterion='gini',class_weight={0:.3, 1:.7})
	clf2_2 = RandomForestClassifier(random_state=RANDOM_SEED,n_estimators=50,criterion='entropy',class_weight={0:.3, 1:.7})
	clf12 = ExtraTreesClassifier(random_state=RANDOM_SEED,n_estimators=50,criterion='gini',class_weight={0:.3, 1:.7})
	clf12_2 = ExtraTreesClassifier(random_state=RANDOM_SEED,n_estimators=50,criterion='entropy',class_weight={0:.3, 1:.7})
	clf10 = GradientBoostingClassifier(learning_rate=0.05, subsample=0.8, max_depth=6, n_estimators=50)
	clf3 = GaussianNB()
	clf4 = SVC(probability=True,kernel="linear",class_weight={0:.3, 1:.7})
	clf6 = GaussianProcessClassifier()
	clf7 = MLPClassifier()
	clf8 = AdaBoostClassifier(base_estimator=SVC(probability=True, kernel='linear',class_weight={0:.3, 1:.7}))
	# clf13 = AdaBoostClassifier(base_estimator=RidgeClassifierCV(),algorithm = 'SAMME')
	clf9 = QuadraticDiscriminantAnalysis()
	# clf13 = RidgeClassifierCV()
	clf14 = SGDClassifier(loss="log")
	lr = LogisticRegression(class_weight={0:.3, 1:.7})
	xgboost_clf = xgb.sklearn.XGBClassifier(n_estimators=1000,subsample=0.8,objective= 'binary:logistic')

	np.random.seed(RANDOM_SEED)
	sclf = StackingCVClassifier(classifiers=[clf1, clf1_2,clf2,clf2_2,clf12,clf12_2,clf10,clf3,clf9,lr,clf4,xgboost_clf,clf7,clf8,clf14], 
								meta_classifier=lr,use_probas = True,use_features_in_secondary = True)
	params = {'logisticregression__C': [0.01,0.1,0.5,1,5,10],'svc__C':[0.01,0.5,10,100]}
	params['xgbclassifier__learning_rate'] = [0.1,0.5,1]
	params['xgbclassifier__max_depth'] = [5,10,20]
	params['xgbclassifier__min_child_weight'] = [1,5,10]
	params['xgbclassifier__gamma'] = [0,0.1,0.3,0.5,0.8]
	params['xgbclassifier__colsample_bytree'] = [0.1,0.5,0.8]
	params['xgbclassifier__max_delta_step'] = [0,0.1,0.4,0.8]
	# params['meta-svc__C'] = [0.01,0.1,0.5,1,5,10]
	params['meta-logisticregression__C'] = [0.01,0.1,0.5,1,5,10]
	## tried many meta-classifiers, meta-logisticregression__C is the best
	# params['meta-xgbclassifier__learning_rate'] = [0.1,0.5,1]
	# params['meta-xgbclassifier__max_depth'] = [5,10,20]
	# params['meta-xgbclassifier__min_child_weight'] = [1,5,10]
	# params['meta-xgbclassifier__gamma'] = [0,0.1,0.3,0.5,0.8]
	# params['meta-xgbclassifier__colsample_bytree'] = [0.1,0.5,0.8]
	# grid = GridSearchCV(estimator=sclf, 
						# param_grid=params, 
						# cv=3,
						# refit=True,n_jobs=-1,scoring="average_precision",verbose=10)
	grid = RandomizedSearchCV(estimator=sclf, 
						param_distributions=params, 
						cv=2,
						n_iter=n_iter,
						refit=True,n_jobs=-1,scoring="average_precision",verbose=10)					

	grid.fit(X_train, y_train)

	# cv_keys = ('mean_test_score', 'std_test_score', 'params')


	print('Best parameters: %s' % grid.best_params_)
	print('CV best avg precision: %.2f' % grid.best_score_)

	best_clf = grid.best_estimator_

	predict_prob = best_clf.predict_proba(X_test)
	predict_prob = np.array(map(lambda x:x[1],predict_prob)).ravel()
	auPRC = scikitlearn_calc_auPRC(y_test, predict_prob)
	auROC = roc_auc_score(y_test, predict_prob)

	Y_pred = best_clf.predict(X_test)
	# print Y_pred
	MCC = matthews_corrcoef(y_test,Y_pred)
	f1 = f1_score(y_test,Y_pred)
	BACC = Balanced_ACC(y_test,Y_pred)


	print('testing auROC: %.2f' % auROC)
	print('testing auPRC: %.2f' % auPRC)
	print('testing MCC: %.2f' % MCC)
	print('testing f1: %.2f' % f1)
	print('testing BACC: %.2f' % BACC)





	return best_clf



















In [6]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix
directory = "./"
properties = pd.read_csv("/home/working/my_second_drive/projects/liyc_own/Zillow/"+'properties_2016.csv')
train = pd.read_csv(directory+"train_2016_v2.csv")
print( "\nProcessing data for XGBoost ...")
for c in properties.columns:
    properties[c]=properties[c].fillna(-1)
    if properties[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(properties[c].values))
        properties[c] = lbl.transform(list(properties[c].values))

train_df = train.merge(properties, how='left', on='parcelid')

train_df["transactiondate"] = pd.to_datetime(train_df["transactiondate"])
train_df["Month"] = train_df["transactiondate"].dt.month

x_train = train_df.drop(['parcelid', 'logerror','transactiondate'], axis=1)
x_test = properties.drop(['parcelid'], axis=1)

x_test["transactiondate"] = '2016-07-01'
x_test["transactiondate"] = pd.to_datetime(x_test["transactiondate"])
x_test["Month"] = x_test["transactiondate"].dt.month #should use the most common training date 2016-10-01
x_test = x_test.drop(['transactiondate'], axis=1)

# shape        
print('Shape train: {}\nShape test: {}'.format(x_train.shape, x_test.shape))

# drop out ouliers
train_df=train_df[ train_df.logerror > -0.4 ]
train_df=train_df[ train_df.logerror < 0.419 ]
x_train=train_df.drop(['parcelid', 'logerror','transactiondate'], axis=1)
y_train = train_df["logerror"].values.astype(np.float32)
x_train = x_train.values.astype(np.float32, copy=False)
x_test = x_test.values.astype(np.float32, copy=False)  



Processing data for XGBoost ...
Shape train: (90275, 58)
Shape test: (2985217, 58)


In [7]:
x_train

array([[  1.00000000e+00,  -1.00000000e+00,  -1.00000000e+00, ...,
         -1.00000000e+00,   6.03710669e+13,   1.00000000e+00],
       [ -1.00000000e+00,  -1.00000000e+00,  -1.00000000e+00, ...,
         -1.00000000e+00,  -1.00000000e+00,   1.00000000e+00],
       [  1.00000000e+00,  -1.00000000e+00,  -1.00000000e+00, ...,
         -1.00000000e+00,   6.03746363e+13,   1.00000000e+00],
       ..., 
       [ -1.00000000e+00,  -1.00000000e+00,  -1.00000000e+00, ...,
         -1.00000000e+00,   6.03730089e+13,   1.20000000e+01],
       [ -1.00000000e+00,  -1.00000000e+00,  -1.00000000e+00, ...,
          1.40000000e+01,   6.03743259e+13,   1.20000000e+01],
       [ -1.00000000e+00,  -1.00000000e+00,  -1.00000000e+00, ...,
         -1.00000000e+00,   6.03760120e+13,   1.20000000e+01]], dtype=float32)

In [ ]:
model = mixer_reg(x_train,y_train,n_iter=2)

Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV] ridge__alpha=10, lasso__alpha=1, meta-lasso__alpha=10.0 .........
[CV] ridge__alpha=10, lasso__alpha=1, meta-lasso__alpha=10.0 .........
[CV] ridge__alpha=5, lasso__alpha=10, meta-lasso__alpha=0.1 ..........
[CV] ridge__alpha=5, lasso__alpha=10, meta-lasso__alpha=0.1 ..........


/usr/local/lib/python2.7/dist-packages/scipy/linalg/basic.py:223: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number: 7.1712605472e-12
  ' condition number: {}'.format(rcond), RuntimeWarning)
/usr/local/lib/python2.7/dist-packages/scipy/linalg/basic.py:223: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number: 1.04259705036e-10
  ' condition number: {}'.format(rcond), RuntimeWarning)
/usr/local/lib/python2.7/dist-packages/scipy/linalg/basic.py:223: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number: 6.65447558168e-11
  ' condition number: {}'.format(rcond), RuntimeWarning)
/usr/local/lib/python2.7/dist-packages/scipy/linalg/basic.py:223: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be

[CV]  ridge__alpha=5, lasso__alpha=10, meta-lasso__alpha=0.1, score=-0.0536348380935, total=71.6min


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 71.7min


[CV]  ridge__alpha=5, lasso__alpha=10, meta-lasso__alpha=0.1, score=-0.0526281792432, total=72.7min


[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed: 72.7min remaining: 72.7min


[CV]  ridge__alpha=10, lasso__alpha=1, meta-lasso__alpha=10.0, score=-0.0536348380935, total=90.1min
[CV]  ridge__alpha=10, lasso__alpha=1, meta-lasso__alpha=10.0, score=-0.0526281792432, total=90.9min


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed: 90.9min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed: 90.9min finished
/usr/local/lib/python2.7/dist-packages/scipy/linalg/basic.py:223: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number: 2.3032038516e-12
  ' condition number: {}'.format(rcond), RuntimeWarning)
/usr/local/lib/python2.7/dist-packages/scipy/linalg/basic.py:223: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number: 7.55953077913e-11
  ' condition number: {}'.format(rcond), RuntimeWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/usr/local/lib/p